More linear models: multiple factors, stepwise selection, binomial response variables

In [ ]:
library(mosaic)
library(car)
library(ggplot2)
library(MASS)
library(plyr)
library(emmeans)

To reduce variation in the error, we can include a block variable like age, plot, sex, litter, or location.

Example from the book, 12.3. 

*Note: I am reading this data in manually because this works better for the Jupyter notebook but you can just use the algae12.xlxs data set imported into R*

In [ ]:
Lake<-c("A","B","C","D","A","B","C","D","A","B","C","D")
CC<-c(425,500,100,325,139,215,30,100,56,115,10,28)
Depth<-c("Surface","Surface","Surface","Surface", "m1", "m1", "m1", "m1", "m3", "m3", "m3", "m3")

D1<-cbind.data.frame(Lake,CC)
algae<-cbind.data.frame(D1,Depth)
algae

Does algae chlorophyll concentration (CC) differ between depths? We control for lake

In [ ]:
lm1<-lm(CC~Depth+Lake, data=algae)
anova(lm1)

We find that chlorophyll concentration does differ between depth (p=0.003) after accounting for variation in lakes (which is also significant p=0.035).

In [ ]:
ggplot(algae, aes(x=Depth, y=CC,color=Lake)) + geom_point(size=5) +theme(panel.border = element_blank(), panel.grid.major = element_blank(), panel.grid.minor = element_blank(), axis.line = element_line(colour = "black"), axis.text=element_text(size=24), axis.title=element_text(size=24,face="bold"))

Two-way factorial ANOVA

In a two-way ANOVA, we have two factors (rather than a factor and a block) and we can look at interactions between these factors. Interactions are coded as "*" or ":"
Example from the book, 12.6. 

In [ ]:
Age<-c("A","A","A","A","A","M","M","M","M","M","O","O","O","O","O")
Sex<-c(rep("M", 15))
BP<-c(108,110,90,80,100,120,125,130,120,130,145,150,130,155,140)
       
D1<-cbind.data.frame(Sex,BP)
Males<-cbind.data.frame(D1,Age)
Males

In [ ]:
Sex2<-c(rep("F",15))
BP2<-c(110,105,100,90,102,110,105,115,100,120,130,125,135,130,120)

D2<-cbind.data.frame(Sex2,BP2)
Females<-cbind.data.frame(D2,Age)
colnames(Females)<-c("Sex","BP","Age")
Females

In [ ]:
Hamster<-rbind.data.frame(Males,Females)
Hamster

In [ ]:
mod2<-lm(BP~Age+Sex+Age:Sex,data=Hamster)
anova(mod2)

There is a significant interaction between Age and Sez (p=0.0239). Let's visualize it.

In [ ]:
ggplot(Hamster, aes(x=Age, y=BP,fill=Sex)) + geom_boxplot() +theme(panel.border = element_blank(), panel.grid.major = element_blank(), panel.grid.minor = element_blank(), axis.line = element_line(colour = "black"), axis.text=element_text(size=24), axis.title=element_text(size=24,face="bold"))

We can see that males have a stronger effect of age than females. The sexes are not different at adolescence ("A") but are very different when old ("O")

Stepwise model selection

In [ ]:
summary(iris)

In [ ]:
fullmod<-lm(Sepal.Length~Sepal.Width+Species+ Species:Sepal.Width,data=iris)
shapiro.test(resid(fullmod))
qqPlot(fullmod, simulate=F)
plot(fullmod)

Because the residuals are not normal, lets try a transformation

In [ ]:
iris$RootSepLen<-sqrt(iris$Sepal.Length)
summary(iris)

In [ ]:
fullmod2<-lm(RootSepLen~Sepal.Width+Species+ Species:Sepal.Width,data=iris)
shapiro.test(resid(fullmod2))
qqPlot(fullmod2, simulate=F)
plot(fullmod2)

That helped, so let's continue with our transformed sqare-root model and do model selection

In [ ]:
step<-stepAIC(fullmod2)
step$anova

We conclude we should removed the interaction from the model and our final model is just Sepal.Width and Species

In [ ]:
bestmod<-lm(RootSepLen~Sepal.Width+Species,data=iris)
Anova(bestmod,type=3)
lsmeans(bestmod, pairwise ~ Species,adjust = "Tukey")

In [ ]:
ggplot(iris, aes(x=Sepal.Width, y=RootSepLen,color=Species)) + geom_point(size=5) +theme(panel.border = element_blank(), panel.grid.major = element_blank(), panel.grid.minor = element_blank(), axis.line = element_line(colour = "black"), axis.text=element_text(size=24), axis.title=element_text(size=24,face="bold"))

Generalized linear models for binomial variables (logistic regression)

Used when our response is a yes/no variable following a binomial distribution

Example: British Election Panel Study we will look at what is associated with voting for the Labour party 1= Labour voted, 0=Vote for other party
We will test is this is related to age or gender

In [ ]:
data(BEPS)
BEPS$Labour<-ifelse(BEPS$vote=="Labour",1,0)
summary(BEPS)

In [ ]:
mod5<-glm(Labour~age+ gender,data=BEPS)
summary(mod5)
anova(mod5)

Only age is significant. Let's look at the proportion voting for labour or not relative to age. 
To plot, we will break the data up using the cut function and summarize using ddply to get the proportion that vote labour in each age group.

In [ ]:
AGroup<-c(20,45,70,95)
BEPS$Age_Group<-cut(BEPS$age,breaks=AGroup)
Sum.BEPS<-ddply(BEPS, c("Age_Group"),summarise, numVote=sum(Labour), total=sum(!is.na(Labour)), PropVote=numVote/total)
Sum.BEPS

Now we can plot the proprotion that vote labour for different groups

In [ ]:
ggplot(Sum.BEPS, aes(x=Age_Group, y=PropVote))+geom_bar(stat="identity")+theme(panel.border = element_blank(), panel.grid.major = element_blank(), panel.grid.minor = element_blank(), axis.line = element_line(colour = "black"), axis.text=element_text(size=24), axis.title=element_text(size=24,face="bold"))